# ECO475 Group 2 Notebook Code

### Author: Shih-Chieh Lee, Lingyun Ma, Yuwen Zhao

# 1. Basic Setting

## a. Package Install

In [ ]:
#!pip install stats-can
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install statsmodels
#!pip install linearmodels
#!pip install tabula-py #Note: Pls install tabula-py, not tabula——血的教训
#!pip install warnings

## b. Package Import 

In [1]:
# Data Collection Packages
from stats_can import StatsCan #read StatsCan data 


sc = StatsCan(data_folder="/Users/changanlee/Documents/GitHub/Housing_Price_Immigration/Input") 
#Create an instance of StatsCan class

In [2]:
# Import tabula and check java environment
from tabula.io import read_pdf  #Scrape table from pdf files
import requests 
from datetime import datetime
import calendar
import re

In [3]:
# Data Processing Packages
import pandas as pd #pandas
import numpy as np 
import matplotlib.pyplot as plt #data visualization
%matplotlib inline
# activate plot theme
import qeds

In [4]:
# Stats Model Packages
import statsmodels.api as sm # statistical model
from statsmodels.iolib.summary2 import summary_col # summary table for regression result
from linearmodels.iv import IV2SLS # IV 

In [5]:
# Silence all the warnings cuz they're absolutely annoying if you loop it multiple times
import warnings
#warnings.filterwarnings('ignore')

# 2. Data Collection

## A. StatsCan Data 

Let's start with datasets from Statistics Canada, as it's earier to collect directly using StatsCan library

In [92]:
CMA_pop = sc.table_to_df("17-10-0142-01")
CMA_pop = CMA_pop[["REF_DATE","GEO","VALUE"]]
CMA_pop['REF_DATE'] = pd.to_datetime(CMA_pop['REF_DATE'])

CMA_pop = CMA_pop[CMA_pop['GEO'].str.contains('\\(')]
CMA_pop[['GEO', 'Province']] = CMA_pop['GEO'].str.split(',', expand=True, n=1)
CMA_pop['GEO'] = CMA_pop['GEO'].str.strip()
CMA_pop['Province'] = CMA_pop['Province'].str.strip()
CMA_pop['GEO'] = CMA_pop['GEO'].str.replace(r'\s*\(.*\)', '', regex=True)


CMA_pop['REF_DATE']= CMA_pop['REF_DATE'].dt.year.astype(int)
CMA_pop = CMA_pop[CMA_pop['REF_DATE'] >= 2016]
CMA_pop = CMA_pop.rename(columns = {"REF_DATE":"Year"})
CMA_pop

,Year,GEO,VALUE,Province
78033,2016,St. John's,111467.0,Newfoundland and Labrador
78034,2016,Conception Bay South,26810.0,Newfoundland and Labrador
78035,2016,Mount Pearl,23596.0,Newfoundland and Labrador
78036,2016,Paradise,22023.0,Newfoundland and Labrador
78037,2016,Corner Brook,20155.0,Newfoundland and Labrador
...,...,...,...,...
114439,2022,Baffin,0.0,"Unorganized (NO), Nunavut"
114440,2022,Keewatin,0.0,"Unorganized (NO), Nunavut"
114441,2022,Bathurst Inlet,0.0,Nunavut
114442,2022,Umingmaktok,0.0,Nunavut


In [93]:
CMA_ON_BC = CMA_pop[CMA_pop["Province"].isin(["British Columbia", "Ontario"])]
CMA_ON_BC.reset_index(drop = True)

,Year,GEO,VALUE,Province
0,2016,Toronto,2819399.0,Ontario
1,2016,Ottawa,964341.0,Ontario
2,2016,Mississauga,746352.0,Ontario
3,2016,Brampton,617571.0,Ontario
4,2016,Hamilton,552272.0,Ontario
...,...,...,...,...
9039,2022,Nedoats 11,0.0,British Columbia
9040,2022,Babine Lake 21B,0.0,British Columbia
9041,2022,Mission Lands 17,0.0,British Columbia
9042,2022,Good Hope Lake,0.0,British Columbia


In [94]:
CMA_ON = CMA_ON_BC[CMA_ON_BC["Province"] == "Ontario"]

## B. CREA Monthly House Price Index Data

### 1) Define Functions

In [36]:
# Define func to generate URLs based on months and years
def generate_url(month, year, cma):
    if cma == "Toronto":
        if month < 10:
            return f"https://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_0{month}{year}.pdf"
        else:
            return f"https://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_{month}{year}.pdf"
            #return the corresponding GTA HPI monthly report pdf for scrapping
    elif cma == "Vancouver":
        url = generate_url_van(month, year)
        return url

### 1) GTA Data

#### a) Define Functions

In [8]:
# Define func to extract table from pdf monthly report & data cleaning
def extract_table(url, month, year):

    # Extract table from pdf
    tables = read_pdf(url, pages="2", lattice = "True", multiple_tables = "False", 
                      area = [10,0,97,100],relative_area = "True", silent = "True") 
        # Note: lattice should be set as True for our case 
            #to read everything on page 2 as one table
            
    table = tables[0]
    
    # Clean the data table
    new_header = table.iloc[0] # Set new header with the first row of the table
    table = table[1:]  # Take the data below the header row
    table.columns = new_header  # Set the new header
    
    # We only want data about composite / residential property, which is the first four columns
    table = table.iloc[:, :4]
    
    # Rename the first column to "Location" 
    table = table.rename(columns={table.columns[0]: "Location"})

    # New Column for Month-Year
    if month < 10:
        table["Month_Year"] = f'0{month}_20{year}'
    else:
        table["Month_Year"] = f'{month}_20{year}'
        
    table.reset_index(drop=True, inplace=True)
    
    return table

#### b) Loop to Scrape Data

In [ ]:
data_list = []
cma = "Toronto"

for year in range(16, 23):
    if year != 22:
        for month in range(1, 13):
            url = generate_url(month, year, cma)
            extracted_data = extract_table(url, month, year)
        
            if extracted_data is not None:
                extracted_data.reset_index(drop = True, inplace = True)
                data_list.append(extracted_data)

    else:
        for month in range(1, 6):
            url = generate_url(month, year)
            extracted_data = extract_table(url, month, year)
        
            if extracted_data is not None:
                extracted_data.reset_index(drop = True, inplace = True)
                data_list.append(extracted_data)
            
HPI_GTA = pd.concat(data_list, ignore_index = True)

In [ ]:
HPI_GTA.to_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GTA.csv",
               index = False)

For some pdf formatting reasons that I cannot solve right now, the last monthly report that can be extracted using read_pdf is May 2022. May need to mannually extract the rest of the data. See example below:

In [ ]:
#Take June 2022 for example

url = generate_url(6,22) 
tables = read_pdf(url, pages = "2", lattice = "True", multiple_tables = "False", area = [10,0,97,100],relative_area = "True")
tables[0].head()

The column heading format is correct, but the value it reads is completely nonsense.

Whatever... Let's move on to GTA data first

### 2) GVA Data

First of all, Jan. 2016 ~ July 2016 monthly report is missing...

Second, GVA report url has multiple formats over time:

https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-August-2016.pdf

https://www.gvrealtors.ca/content/dam/rebgv_org_content/monthly-market-reports/2018-Dec-stats-pkg.pdf

https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/marketwatch/monthly_reports/REBGV-Stats-Pkg-November-2019-F.pdf

OK, let's try accessing the report using all 3 formats for all month-year combination and see how it goes

In [39]:
def generate_url_van(month, year):

    # Define the base URLs for each format
    base_urls = [
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/monthly-market-reports/{Year}-{Month}-stats-pkg.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/marketwatch/monthly_reports/REBGV-Stats-Pkg-{Month}-{Year}-F.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Package-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-{Month}-{Year}-Updated%20HPI.pdf",
        "https://members.rebgv.org/news/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{YearMonth}-REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV%20Stats%20Package%20{Month}%20{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{Year}-01-{Month}-Stats-Package.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{Year}-01-{Month}-Stats--Package.pdf"
    ]


    # Get the month name and abbreviation
    year = 2000 + year
    month_name = calendar.month_name[month]
    month_abbr = calendar.month_abbr[month]
    year_month = f"{year}{month:02d}"

    # Iterate through each URL pattern
    for base_url in base_urls:
        # Replace placeholders in the URL pattern
        url_1 = base_url.format(Year=str(year), Month=month_name, YearMonth=year_month)
        url_2 = base_url.format(Year=str(year), Month=month_abbr, YearMonth=year_month)

        # Try fetching the first URL
        if requests.get(url_1).status_code == 200:
            return url_1

        # Try fetching the second URL if the first one fails
        elif requests.get(url_2).status_code == 200:
            return url_2

        # Special case for September URLs
        if month == 9:
            url_3 = base_url.format(Year=str(year), Month="Sept", YearMonth=year_month)
            if requests.get(url_3).status_code == 200:
                return url_3

In [42]:
def extract_table_van(url, month, year):    
    tables = read_pdf(url, pages = "3", multiple_tables = "False")
    
    if not tables or tables[0].empty:
        return None
    
    else:
        table = tables[0]
        new_header = table.iloc[1] # Set new header with the second row of the table
        new_header[0] = "Property Type"
        new_header[1] = "Location"
        table = table[2:]  # Take the data below the header row
        table.columns = new_header

        end_here_index = (table['Property Type'] == 'Single Family Detached').idxmax()
        table = table.iloc[:end_here_index, 1:]

        if month < 10:
            table["Month_Year"] = f'0{month}_{year}'
        else:
            table["Month_Year"] = f'{month}_{year}'

    return table

In [38]:
data_list = []
cma = "Vancouver"

for year in range(16, 24):
    for month in range (1,13):
        
        url = generate_url(month, year, "Vancouver")
        if url is not None:
            extracted_data = extract_table_van(url, month, year)

            if extracted_data is not None:
                extracted_data.reset_index(drop=True, inplace=True)
                data_list.append(extracted_data)
                print("Collected", month,"-",year, " data.")

HPI_GVA = pd.concat(data_list, ignore_index=True)

columns_name = ["Location", "Price", "Index", "%1M_Change", "%3M_Change", "%6M_Change", 
               "%1Y_Change", "%3Y_Change", "%5Y_Change", "%10Y_Change", "Month_Year"]

HPI_GVA.columns = columns_name

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 3 - 16  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 4 - 16  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


...

Collected 9 - 23  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 10 - 23  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 11 - 23  data.


In [ ]:
HPI_GVA.to_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GVA.csv",
               index = False)

### 3) Append Dataset

In [160]:
HPI_GTA = pd.read_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GTA_complete.csv")
HPI_GTA = HPI_GTA.iloc[:,:5].drop(columns=["Yr./Yr. % Chg."])
HPI_GTA["Region"] = "Greater Toronto Area"

In [29]:
HPI_GVA = pd.read_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GVA.csv")
HPI_GVA = HPI_GVA.drop(columns=["%1M_Change","%3M_Change","%6M_Change","%1Y_Change", "%3Y_Change", "%5Y_Change", "%10Y_Change"])
HPI_GVA = HPI_GVA.rename(columns={"Price":"Benchmark"})
HPI_GVA["Region"] = "Greater Vancouver Area"
HPI_GVA

,Location,Benchmark,Index,Month_Year,Region
0,Lower Mainland,"$719,500",202.9,03_2016,Greater Vancouver Area
1,Greater Vancouver,"$815,000",213.3,03_2016,Greater Vancouver Area
2,Bowen Island,"$653,500",141.7,03_2016,Greater Vancouver Area
3,Burnaby East,"$759,000",209.5,03_2016,Greater Vancouver Area
4,Burnaby North,"$675,000",202.9,03_2016,Greater Vancouver Area
...,...,...,...,...,...
1915,Vancouver West,"$1,325,900",311.7,11_2023,Greater Vancouver Area
1916,West Vancouver,"$2,560,500",276.0,11_2023,Greater Vancouver Area
1917,Whistler,"$1,388,200",311.9,11_2023,Greater Vancouver Area
1918,Lower Mainland,"$1,764,500",386.4,11_2023,Greater Vancouver Area


In [34]:
HPI_Overall = pd.concat([HPI_GTA, HPI_GVA],ignore_index = True)
HPI_Overall

,Location,Index,Benchmark,Month_Year,Region
0,TREB Total,190.4,"$581,100",01_2016,Greater Toronto Area
1,Halton Region,206.6,"$675,800",01_2016,Greater Toronto Area
2,Burlington,202.9,"$601,200",01_2016,Greater Toronto Area
3,Halton Hills,184.5,"$537,300",01_2016,Greater Toronto Area
4,Milton,210.4,"$580,500",01_2016,Greater Toronto Area
...,...,...,...,...,...
5563,Vancouver West,311.7,"$1,325,900",11_2023,Greater Vancouver Area
5564,West Vancouver,276.0,"$2,560,500",11_2023,Greater Vancouver Area
5565,Whistler,311.9,"$1,388,200",11_2023,Greater Vancouver Area
5566,Lower Mainland,386.4,"$1,764,500",11_2023,Greater Vancouver Area


# 3. Merge Dataset

## A. Test

In [162]:
HPI_GTA.head()
HPI_GTA[["Month","Year"]] = HPI_GTA['Month_Year'].str.split('_', expand=True, n=1)
HPI_GTA_raw = HPI_GTA

,Location,Index,Benchmark,Month_Year,Region,Month,Year
0,TREB Total,190.4,"$581,100",01_2016,Greater Toronto Area,01,2016
1,Halton Region,206.6,"$675,800",01_2016,Greater Toronto Area,01,2016
2,Burlington,202.9,"$601,200",01_2016,Greater Toronto Area,01,2016
3,Halton Hills,184.5,"$537,300",01_2016,Greater Toronto Area,01,2016
4,Milton,210.4,"$580,500",01_2016,Greater Toronto Area,01,2016
...,...,...,...,...,...,...,...
3643,Adjala-Tosorontio,381.2,"$1,062,300",12_2023,Greater Toronto Area,12,2023
3644,Bradford West,402.4,"$1,094,900",12_2023,Greater Toronto Area,12,2023
3645,EGswsiallimbury,378.3,"$769,500",12_2023,Greater Toronto Area,12,2023
3646,Innisfil,378.9,"$772,100",12_2023,Greater Toronto Area,12,2023


In [173]:
HPI_GTA['Location'] = HPI_GTA['Location'].str.replace(r'Region|City of |Township of ', '', regex=True)

#HPI_GTA.loc[HPI_GTA['Location'] == "Halton"] = "Halton Hills"
HPI_GTA.loc[HPI_GTA['Location'] == "Bradford West", "Location"] = "Bradford West Gwillimbury"
HPI_GTA.loc[HPI_GTA['Location'] == "GEswsiallimbury", 'Location'] = "East Gwillimbury"
HPI_GTA.loc[HPI_GTA['Location'] == "EGswsiallimbury", 'Location'] = "East Gwillimbury"

HPI_GTA['Location'] = HPI_GTA['Location'].str.strip()

HPI_GTA = HPI_GTA[HPI_GTA["Year"] !="2023"]

/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_53164/3222062664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HPI_GTA['Location'] = HPI_GTA['Location'].str.replace(r'Region|City of |Township of ', '', regex=True)
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_53164/3222062664.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HPI_GTA['Location'] = HPI_GTA['Location'].str.strip()


In [77]:
CMA_ON.head()

,REF_DATE,GEO,VALUE,Province
80186,2016-01-01,Toronto,2819399.0,Ontario
80187,2016-01-01,Ottawa,964341.0,Ontario
80188,2016-01-01,Mississauga,746352.0,Ontario
80189,2016-01-01,Brampton,617571.0,Ontario
80190,2016-01-01,Hamilton,552272.0,Ontario


In [174]:
CMA_ON["Year"] = CMA_ON["Year"].astype(str)
merged_df_ON = HPI_GTA.merge(CMA_ON, how="left", left_on=["Location", "Year"], right_on=["GEO", "Year"])

/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_53164/2749079350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CMA_ON["Year"] = CMA_ON["Year"].astype(str)


In [177]:
Unmerged_dataset = merged_df_ON[merged_df_ON["GEO"].isna()]


# Find rows where "Location" matches the specified values
rows_to_drop = Unmerged_dataset[Unmerged_dataset["Location"].isin(["Halton", "Peel", "York", "Durham", "Dufferin County", "Simcoe County"])].index

# Drop these rows
Unmerged_dataset = Unmerged_dataset.drop(rows_to_drop)
Unmerged_dataset

,Location,Index,Benchmark,Month_Year,Region,Month,Year,GEO,VALUE,Province
0,TREB Total,190.4,"$581,100",01_2016,Greater Toronto Area,01,2016,NaN,NaN,NaN
38,TREB Total,193,"$589,000",02_2016,Greater Toronto Area,02,2016,NaN,NaN,NaN
76,TREB Total,196.4,"$599,400",03_2016,Greater Toronto Area,03,2016,NaN,NaN,NaN
114,TREB Total,201.4,"$614,700",04_2016,Greater Toronto Area,04,2016,NaN,NaN,NaN
152,TREB Total,208.3,"$635,700",05_2016,Greater Toronto Area,05,2016,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3002,TREB Total,355.4,"$1,124,600",08_2022,Greater Toronto Area,08,2022,NaN,NaN,NaN
3040,TREB Total,351,"$1,110,700",09_2022,Greater Toronto Area,09,2022,NaN,NaN,NaN
3078,TREB Total,347.1,"$1,098,200",10_2022,Greater Toronto Area,10,2022,NaN,NaN,NaN
3116,TREB Total,344.4,"$1,089,800",11_2022,Greater Toronto Area,11,2022,NaN,NaN,NaN


In [178]:
12*(2022-2016+1)

84